In [1]:
import configparser
import dataclasses
import os
import re
import shutil
import statistics
from collections import Counter
from itertools import chain
from pathlib import Path
from time import perf_counter

import pandas as pd
from Bio import SeqIO, pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
import logging
import xlsxwriter

import utils
from fns import *

## START LOGGING 

In [2]:
Path('logs').mkdir(parents=True, exist_ok=True)
log = utils.make_logger("logs/rp_log")
log.info("\nSTART")
log.info("\nCHECKING CONFIGURATION FILE CONTENTS...")

## GET PARAMS FROM CONFIG FILE

In [3]:
# checking config file
# if any error occurs then program terminates; then check the config file
config = configparser.ConfigParser()
config.read_file(open('config.ini'))

abi_sequence_folder = checking_dirs(config['Paths']['abi_sequence_folder'], log, log_msg=True, create_dir=False)
vh_template_sequence_folder = checking_dirs(config['Paths']['vh_template_sequence_folder'], log, log_msg=True, create_dir=False)
vl_template_sequence_folder = checking_dirs(config['Paths']['vl_template_sequence_folder'], log, log_msg=True, create_dir=False)
results_dir = checking_dirs(config['Paths']['results_dir'], log, log_msg=True, create_dir=True)
h3_nt_data_sheet_filepath = check_files(config['Files']['h3_nt_data_sheet_filepath'], log)
df = check_tsv_file(h3_nt_data_sheet_filepath, log)
excel_path_file_name = create_results_excel_file_path(config['Paths']['results_dir'], config['Files']['output_excel_file_name'])

# patterns
pat_vh, pat_vl = get_patterns(config)
# patterns to remove from abi names
patrm_vh_abi, patrm_vl_abi = get_patterns_to_rm(config)
# patterns to remove from genbank names
patrm_vh_gb, patrm_vl_gb = get_patterns_to_rm_from_genbank(config)

# get alignment parameters
par_match, par_missmatch, par_open, par_extend, par_filter_thresh = get_alignment_params(config)

# get alignment start and end regions
vh_seq_start_a, vh_seq_end_a, vl_seq_start_a, vl_seq_end_a = get_alignment_seq_start_and_end(config)

# creating dirs for copying matched abi files
res_dir_vh = checking_dirs(f"{results_dir}/{pat_vh}", log, log_msg=True, create_dir=True)
res_dir_vl = checking_dirs(f"{results_dir}/{pat_vl}", log, log_msg=True, create_dir=True)


## GET ABI FILES

In [4]:
log.info("\nCHECKING THE AB1 FILES...")
vh_abi_dict = {i.name.replace('.abi', '').replace(patrm_vh_abi, '') : str(i) for i in sorted([*Path(abi_sequence_folder).glob(f"*{pat_vh}*.abi")])}
vl_abi_dict = {i.name.replace('.abi', '').replace(patrm_vl_abi, '') : str(i) for i in sorted([*Path(abi_sequence_folder).glob(f"*{pat_vl}*.abi")])}

if len(vh_abi_dict) == len(vl_abi_dict):
    log.info(f"[+] INFO: There are {len(vh_abi_dict)} abi files in the abi sequence folder")
else:
    log.warning(f"[+] WARN: There is a differrence in number of abi files in vh: {len(vh_abi_dict)} and vl {len(vl_abi_dict)}")

In [5]:
sample_ids = checking_ab1_files(log, vh_abi_dict, vl_abi_dict)

In [6]:
log.info("\nCHECKING THE GENBANK FILES...")
vh_template_gb_dict =  {i.name.replace('.gb', '') : str(i) for i in sorted(Path(vh_template_sequence_folder).glob('*.gb'))}
vl_template_gb_dict =  {i.name.replace('.gb', '') : str(i) for i in sorted(Path(vl_template_sequence_folder).glob('*.gb'))}

if len(vh_template_gb_dict) == len(vl_template_gb_dict):
    log.info(f"[+] INFO: There are {len(vh_template_gb_dict)} genbank files in each dir vh and vl")
else:
    log.warning(f"[+] WARN: There is a differrence in number of genbank files in vh: {len(vh_template_gb_dict)} and vl {len(vl_template_gb_dict)}")

## GET PROBE SEQS INTO A DICTIOANRY

In [7]:
# dictionary containing probe seq and name
h3_dict = df.set_index('name', drop=True).to_dict().get('h3_nt')

## Sanity Check

In [8]:
h3_dict

{'TMH577-hF-014-A01': 'GCGAGAGTGGGGCGAAGCAGTGGCTGGTACTACCTCGACCAC',
 'TMH577-hF-014-A06': 'GCGAAAAGCCGATGGGTTGGAGATAGTAGTGGTTACTACGACTAC',
 'TMH577-hF-014-B02': 'GTCAGAAGTACGGTGGTCAGACAGGGGGCCCCTAATGCTTTTGATCTC',
 'TMH577-hF-014-B03': 'GCGAGAGAGCTTAACTACGGTGGGAACTCACGGGCTTTTGATATC',
 'TMH577-hF-014-B07': 'GCGAAAGAGGAGTATAGCAGCTCGTCAGGACGAGTCTCCCGATACTTTGACTAC',
 'TMH577-hF-014-B12': 'GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACGGTTTCGACGTC',
 'TMH577-hF-014-C05': 'GCGAGAGAGGGTAGCAGCTCGCCACTTGACTAC',
 'TMH577-hF-014-C06': 'GCAATGGAGTGGCTAGGGTTCGAC',
 'TMH577-hF-014-D02': 'GCGAGGGTAAGATTGATGAGTATTAGCAGCTGGAAAAGAGCGTCTGATATG',
 'TMH577-hF-014-D03': 'GCGAAAGGAGGCTAC',
 'TMH577-hF-014-D12': 'GCGAGTGGATATTACGATATTTTGACTGGTTATTATGACTAC',
 'TMH577-hF-014-F01': 'GCGAGAGAGGGCGTATATGACCTCACCTGGTACGGGAACCATGACTTTGACTAC',
 'TMH577-hF-014-F10': 'GCTCAGGTCAACTATGCCCGCTTTGACTTC',
 'TMH577-hF-014-G02': 'GCGTGCCCAGTGGGCCCGAGTTCATGGGCTTTTGATATC',
 'TMH577-hF-014-G03': 'GCAAGAGGAATTCGCCGTATTACTATGGTTCGGG

In [9]:
vh_abi_dict

{'TMH577-hIgG1-013-A10': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A10_GATC-VH60-2617917.abi',
 'TMH577-hIgG1-013-A11': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A11_GATC-VH60-2617917.abi',
 'TMH577-hIgG1-013-A12': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A12_GATC-VH60-2617917.abi',
 'TMH577-hIgG1-013-A1_A01': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A1_A01_GATC-VH60-2617917.abi',
 'TMH577-hIgG1-013-A2_A02': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A2_A02_GATC-VH60-2617917.abi',
 'TMH577-hIgG1-013-A3_A03': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A3_A03_GATC-VH60-2617917.abi',
 'TMH577-hIgG1-013-A4_A04': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A4_A04_GATC-VH60-2617917.abi',
 'TMH577-hIgG1-013-A5_A05': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1

In [10]:
vl_abi_dict

{'TMH577-hIgG1-013-A10': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A10_VL79.abi',
 'TMH577-hIgG1-013-A11': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A11_VL79.abi',
 'TMH577-hIgG1-013-A12': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A12_VL79.abi',
 'TMH577-hIgG1-013-A1_A01': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A1_A01_VL79.abi',
 'TMH577-hIgG1-013-A2_A02': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A2_A02_VL79.abi',
 'TMH577-hIgG1-013-A3_A03': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A3_A03_VL79.abi',
 'TMH577-hIgG1-013-A4_A04': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A4_A04_VL79.abi',
 'TMH577-hIgG1-013-A5_A05': '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A5_A05_VL79.abi',
 'TMH577-hIgG1-013-A6_A06': '/Users/rp/Desktop/pur

In [11]:
len(h3_dict), len(vh_abi_dict), len(vl_abi_dict)

(80, 186, 186)

## FINDING THE PROBES THAT MATCH VH AND CORRESPONDING VL FILES

In [12]:
def find_match_on_all_h3probes_v2(log: logging, h3_dict: dict,  d: SeqIO.SeqRecord, sample:str, vh_abi_dict: dict, vl_abi_dict: dict) -> list:
    """
    Returns: [orientation of match, h3_probe_name, sample_id, vh_abi_dp, vl_abi_fp, probe_seq, seq_record, abi_seq, trimmed_seq, trimmed_seq_qual]
    inputs
    log: logger 
    h3_dict: dictionary containing h3_name and probe_name
    d: Sequence Record from the abi file
    sample: sample id - comes from sample_ids
    vh_abi_dict: dictionary containing vh names and full file path
    vl_abi_dict: dictionary containing vl names and full file path
    """
    results = []
    for h3key, h3val in h3_dict.items():
        d_trimmed, d_trimquallst = get_trimmed_seq_record(d, get_quality=True)
        seq_f = get_seq_from_record(d, reverse=False)
        seq_r = get_seq_from_record(d, reverse=True)
        vh_fn = vh_abi_dict.get(sample)
        vl_fn = vl_abi_dict.get(sample)
        if h3val in seq_f:
            log.info(f"[+] PROBE_MATCHING: Forward_Strand_Match|{h3key}|{sample}|{vh_fn}|{vl_fn}")
            results.append(["Forward_Strand_Match", h3key, sample, vh_fn, vl_fn, h3val, d, seq_f, d_trimmed, d_trimquallst])
        elif h3val in seq_r:
            log.info(f"[+] PROBE_MATCHING: Reverse_Strand_Match|{h3key}|{sample}|{vh_fn}|{vl_fn}")
            results.append(["Reverse_Strand_Match", h3key, sample, vh_fn, vl_fn, h3val, d, seq_r, d_trimmed, d_trimquallst])
        else:
            pass
    return(results)


def find_match_on_all_h3probes_v3(log: logging, h3_dict: dict,  vh_sr: SeqIO.SeqRecord, vl_sr: SeqIO.SeqRecord, sample:str, vh_abi_dict: dict, vl_abi_dict: dict) -> list:
    """
    If vh matches then get the corresponding vl also
    Returns: [orientation of match, h3_probe_name, sample_id, vh_abi_dp, vl_abi_fp, probe_seq, vh_seq_record, vl_seq_record, vh_trimmed_seq, vh_trimmed_seq_qual, vl_trimmed_seq, vl_trimmed_seq_qual]
    inputs
    log: logger 
    h3_dict: dictionary containing h3_name and probe_name
    d: Sequence Record from the abi file
    sample: sample id - comes from sample_ids
    vh_abi_dict: dictionary containing vh names and full file path
    vl_abi_dict: dictionary containing vl names and full file path
    """
    results = []
    for h3key, h3val in h3_dict.items():
        vh_sr_trimmed, vh_sr_trimquallst = get_trimmed_seq_record(vh_sr, get_quality=True)
        vl_sr_trimmed, vl_sr_trimquallst = get_trimmed_seq_record(vl_sr, get_quality=True)
        
        vh_sr_seq_f = get_seq_from_record(vh_sr, reverse=False)
        vl_sr_seq_f = get_seq_from_record(vl_sr, reverse=False)
        vh_sr_seq_r = get_seq_from_record(vh_sr, reverse=True)
        vl_sr_seq_r = get_seq_from_record(vl_sr, reverse=True)

        vh_fn = vh_abi_dict.get(sample)
        vl_fn = vl_abi_dict.get(sample)
        
        if h3val in vh_sr_seq_f:
            log.info(f"[+] PROBE_MATCHING: Forward_Strand_Match|{h3key}|{sample}|{vh_fn}|{vl_fn}")
            results.append(["Forward_Strand_Match", h3key, sample, vh_fn, vl_fn, h3val, vh_sr, vl_sr, vh_sr_seq_f, vh_sr_trimmed, vh_sr_trimquallst, vl_sr_seq_f, vl_sr_trimmed, vl_sr_trimquallst])
        elif h3val in vh_sr_seq_r:
            log.info(f"[+] PROBE_MATCHING: Reverse_Strand_Match|{h3key}|{sample}|{vh_fn}|{vl_fn}")
            results.append(["Reverse_Strand_Match", h3key, sample, vh_fn, vl_fn, h3val, vh_sr, vl_sr, vh_sr_seq_r, vh_sr_trimmed, vh_sr_trimquallst,  vl_sr_seq_r, vl_sr_trimmed, vl_sr_trimquallst])
                           
        else:
            pass
    return(results)



In [13]:
# # this loop  iternate through all the sample ids and find_match_on_all_h3probes function search for the probe on the normal seq and the rev complemented seq
# # if there is a match which will be returened as a list

# log.info(f"\nITERATING THROUGH EACH SAMPLE ID")
# result_vh, result_vl = [], []
# for sample in sample_ids:
#     # print(f">>{sample}")
#     _vhabi, _vlabi = get_abi_file_path(key=sample, vh_abi_dict=vh_abi_dict, vl_abi_dict=vl_abi_dict)
#     _vh_d = get_seqobj_from_abi(_vhabi)  # returns a seq record obj of VH
#     _vl_d = get_seqobj_from_abi(_vlabi)  # returns a seq record obj of VL
    
#     # matching each probe on  VH and VL - normal and revcomp sequence
#     vh_prob_search = find_match_on_all_h3probes_v2(log, h3_dict, _vh_d, sample, vh_abi_dict, vl_abi_dict)
#     # vl_prob_search = find_match_on_all_h3probes_v2(log, h3_dict, _vl_d, sample, vh_abi_dict, vl_abi_dict)
    
#     if len(vh_prob_search) >=1:
#         result_vh.append(vh_prob_search)
# log.info(f"\nFINISH ITERATING THROUGH EACH SAMPLE ID")
# # if vh and vl ids match then copy the files to a new place
# colnames=["Match","h3_name","sample_id","vh_abi_fp","vl_abi_fp","probe_seq", "seq_record", "abi_seq", "trimmed_seq", "trimmed_seq_qual"]
# df_vh = pd.DataFrame(chain.from_iterable(result_vh))
# df_vh.columns = colnames
# df_vh['vh_abi_filename'] = df_vh.vh_abi_fp.apply(lambda x: x.split("/")[-1])
# df_vh['vl_abi_filename'] = df_vh.vl_abi_fp.apply(lambda x: x.split("/")[-1])

In [14]:
log.info(f"\nITERATING THROUGH EACH SAMPLE ID")
result_vh, result_vl = [], []
for sample in sample_ids:
    # print(f">>{sample}")
    _vhabi, _vlabi = get_abi_file_path(key=sample, vh_abi_dict=vh_abi_dict, vl_abi_dict=vl_abi_dict)
    _vh_d = get_seqobj_from_abi(_vhabi)  # returns a seq record obj of VH
    _vl_d = get_seqobj_from_abi(_vlabi)  # returns a seq record obj of VL
    
    # matching each probe on  VH and VL - normal and revcomp sequence
    vh_prob_search = find_match_on_all_h3probes_v3(log, h3_dict, _vh_d, _vl_d, sample, vh_abi_dict, vl_abi_dict)
    # vl_prob_search = find_match_on_all_h3probes_v2(log, h3_dict, _vl_d, sample, vh_abi_dict, vl_abi_dict)
    
    if len(vh_prob_search) >=1:
        result_vh.append(vh_prob_search)
log.info(f"\nFINISH ITERATING THROUGH EACH SAMPLE ID")
# if vh and vl ids match then copy the files to a new place
colnames=["Match","h3_name","sample_id","vh_abi_fp","vl_abi_fp","probe_seq", "vh_init_sr", "vl_inti_sr","vh_sr_seq_r", "vh_sr_trimmed", "vh_sr_tqlst", "vl_sr_seq_r", "vl_sr_trimmed", "vl_sr_tqlst"]
df_vh = pd.DataFrame(chain.from_iterable(result_vh))
df_vh.columns = colnames
# df_vh['vh_abi_filename'] = df_vh.vh_abi_fp.apply(lambda x: x.split("/")[-1])
# df_vh['vl_abi_filename'] = df_vh.vl_abi_fp.apply(lambda x: x.split("/")[-1])

In [15]:
df_vh

,Match,h3_name,sample_id,vh_abi_fp,vl_abi_fp,probe_seq,vh_init_sr,vl_inti_sr,vh_sr_seq_r,vh_sr_trimmed,vh_sr_tqlst,vl_sr_seq_r,vl_sr_trimmed,vl_sr_tqlst
0,Reverse_Strand_Match,TMH577-hF-012-E03,TMH577-hIgG1-013-A10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGACCCTTTTGGAGCGGCCTAACAAGAGAAAACTACTATTACG...,"(A, T, G, C, C, A, A, A, G, C, C, C, A, A, G, ...","(C, G, G, C, A, T, T, C, T, T, G, A, G, G, C, ...","(T, C, A, C, G, G, G, G, A, T, T, T, C, C, A, ...","(G, G, G, C, C, T, G, C, C, C, C, A, G, A, A, ...","[54, 7, 6, 12, 16, 16, 37, 35, 41, 39, 19, 24,...","(A, A, T, A, T, A, T, A, G, C, A, G, T, T, T, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 18, 20, 53, 53, 42, 36, 25, 30, 52, 58, 5..."
1,Reverse_Strand_Match,TMH577-hF-015-C12,TMH577-hIgG1-013-A11,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCTAGGTACTACTACGGTATGGACGTC,"(C, C, G, T, T, C, C, A, A, A, A, C, C, A, A, ...","(G, G, G, A, A, G, G, C, T, G, A, G, C, A, G, ...","(T, T, A, T, G, G, G, A, C, T, T, T, C, C, T, ...","(T, A, C, T, C, A, C, C, T, G, A, G, C, T, C, ...","[58, 47, 37, 58, 31, 24, 54, 58, 58, 58, 45, 5...","(C, G, C, C, A, A, T, C, A, G, A, A, C, C, C, ...","(A, C, A, G, G, G, A, G, G, G, C, A, G, A, G, ...","[54, 12, 9, 32, 22, 13, 6, 12, 58, 24, 22, 47,..."
2,Reverse_Strand_Match,TMH577-hF-014-B12,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ...","(G, T, G, G, A, A, C, G, T, C, C, T, G, A, G, ...","(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, T, G, G, C, C, C, C, A, G, A, C, G, T, C, ...","[50, 56, 59, 43, 39, 56, 56, 43, 36, 34, 49, 3...","(A, C, G, C, A, G, C, A, C, A, A, A, C, G, A, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 18, 20, 50, 51, 32, 32, 25, 30, 51, 58, 5..."
3,Reverse_Strand_Match,TMH577-hF-015-C08,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ...","(G, T, G, G, A, A, C, G, T, C, C, T, G, A, G, ...","(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, T, G, G, C, C, C, C, A, G, A, C, G, T, C, ...","[50, 56, 59, 43, 39, 56, 56, 43, 36, 34, 49, 3...","(A, C, G, C, A, G, C, A, C, A, A, A, C, G, A, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 18, 20, 50, 51, 32, 32, 25, 30, 51, 58, 5..."
4,Reverse_Strand_Match,TMH577-hF-015-D08,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ...","(G, T, G, G, A, A, C, G, T, C, C, T, G, A, G, ...","(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, T, G, G, C, C, C, C, A, G, A, C, G, T, C, ...","[50, 56, 59, 43, 39, 56, 56, 43, 36, 34, 49, 3...","(A, C, G, C, A, G, C, A, C, A, A, A, C, G, A, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 18, 20, 50, 51, 32, 32, 25, 30, 51, 58, 5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,Reverse_Strand_Match,TMH577-hF-016-G10,TMH577-hIgG1-014-H1_H01,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAAAGGGGGAGCAGCTCGCCCGCTCTACTACGGTATGGACGTC,"(C, C, G, C, C, C, C, A, A, A, G, C, A, A, A, ...","(T, G, G, G, C, A, T, T, C, C, T, G, G, G, G, ...","(T, T, T, G, A, C, T, C, A, C, G, G, G, G, A, ...","(C, C, T, G, A, G, C, T, C, A, C, G, G, T, G, ...","[53, 51, 54, 58, 45, 35, 54, 58, 41, 24, 34, 5...","(A, G, T, A, C, A, T, C, C, A, G, G, G, T, A, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[54, 18, 27, 39, 51, 58, 37, 25, 32, 58, 58, 5..."
179,Reverse_Strand_Match,TMH577-hF-014-B

In [16]:
# if the sahpe of df_vh and df_vl are same and the h3_name in both dataframes are same then copy the matched data to a new dir
log.info(f"[+] INFO: There are {df_vh.shape[0]} matches in df_vh")

## Sanity check

In [17]:
df_vh.head()

,Match,h3_name,sample_id,vh_abi_fp,vl_abi_fp,probe_seq,vh_init_sr,vl_inti_sr,vh_sr_seq_r,vh_sr_trimmed,vh_sr_tqlst,vl_sr_seq_r,vl_sr_trimmed,vl_sr_tqlst
0,Reverse_Strand_Match,TMH577-hF-012-E03,TMH577-hIgG1-013-A10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGACCCTTTTGGAGCGGCCTAACAAGAGAAAACTACTATTACG...,"(A, T, G, C, C, A, A, A, G, C, C, C, A, A, G, ...","(C, G, G, C, A, T, T, C, T, T, G, A, G, G, C, ...","(T, C, A, C, G, G, G, G, A, T, T, T, C, C, A, ...","(G, G, G, C, C, T, G, C, C, C, C, A, G, A, A, ...","[54, 7, 6, 12, 16, 16, 37, 35, 41, 39, 19, 24,...","(A, A, T, A, T, A, T, A, G, C, A, G, T, T, T, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 18, 20, 53, 53, 42, 36, 25, 30, 52, 58, 5..."
1,Reverse_Strand_Match,TMH577-hF-015-C12,TMH577-hIgG1-013-A11,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCTAGGTACTACTACGGTATGGACGTC,"(C, C, G, T, T, C, C, A, A, A, A, C, C, A, A, ...","(G, G, G, A, A, G, G, C, T, G, A, G, C, A, G, ...","(T, T, A, T, G, G, G, A, C, T, T, T, C, C, T, ...","(T, A, C, T, C, A, C, C, T, G, A, G, C, T, C, ...","[58, 47, 37, 58, 31, 24, 54, 58, 58, 58, 45, 5...","(C, G, C, C, A, A, T, C, A, G, A, A, C, C, C, ...","(A, C, A, G, G, G, A, G, G, G, C, A, G, A, G, ...","[54, 12, 9, 32, 22, 13, 6, 12, 58, 24, 22, 47,..."
2,Reverse_Strand_Match,TMH577-hF-014-B12,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ...","(G, T, G, G, A, A, C, G, T, C, C, T, G, A, G, ...","(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, T, G, G, C, C, C, C, A, G, A, C, G, T, C, ...","[50, 56, 59, 43, 39, 56, 56, 43, 36, 34, 49, 3...","(A, C, G, C, A, G, C, A, C, A, A, A, C, G, A, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 18, 20, 50, 51, 32, 32, 25, 30, 51, 58, 5..."
3,Reverse_Strand_Match,TMH577-hF-015-C08,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ...","(G, T, G, G, A, A, C, G, T, C, C, T, G, A, G, ...","(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, T, G, G, C, C, C, C, A, G, A, C, G, T, C, ...","[50, 56, 59, 43, 39, 56, 56, 43, 36, 34, 49, 3...","(A, C, G, C, A, G, C, A, C, A, A, A, C, G, A, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 18, 20, 50, 51, 32, 32, 25, 30, 51, 58, 5..."
4,Reverse_Strand_Match,TMH577-hF-015-D08,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ...","(G, T, G, G, A, A, C, G, T, C, C, T, G, A, G, ...","(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, T, G, G, C, C, C, C, A, G, A, C, G, T, C, ...","[50, 56, 59, 43, 39, 56, 56, 43, 36, 34, 49, 3...","(A, C, G, C, A, G, C, A, C, A, A, A, C, G, A, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 18, 20, 50, 51, 32, 32, 25, 30, 51, 58, 5..."


In [18]:
df_vh.Match.value_counts()

Reverse_Strand_Match    183
Name: Match, dtype: int64

In [19]:
# use this data frame to copy the data to a new place
# use this data frame to match it with genbank
# while matching use the vh_abi_fp and vl_abi_fp to the genbank file

In [20]:
df_vh.shape

(183, 14)

In [21]:
df_vh.head().vh_sr_seq_r.apply(lambda x: len(x))

0    839
1    910
2    447
3    447
4    447
Name: vh_sr_seq_r, dtype: int64

In [22]:
df_vh.head().vh_sr_trimmed.apply(lambda x: len(x))

0    450
1    480
2    239
3    239
4    239
Name: vh_sr_trimmed, dtype: int64

In [23]:
df_vh.head().vh_sr_tqlst.apply(lambda x: len(x))

0    450
1    480
2    239
3    239
4    239
Name: vh_sr_tqlst, dtype: int64

## COPY PROBE MATCHED ABI FILES TO A NEW LOC

In [24]:
log.info(f"\nCOPY MATCHED ABI FILES INTO NEW LOCATION")
res_df_copy = copy_mtched_abi_files_to_resdir(log, res_dir_vh, res_dir_vl, df_vh, log_msg=True)
log.info(f"\nFINISH COPYING FILES")

In [25]:
# this dataframe contain the new and old location of abi files 
res_df_copy.head()

,h3_name,sample_id,abi_out_loc,abi_initial_filepath
0,TMH577-hF-012-E03,TMH577-hIgG1-013-A10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
1,TMH577-hF-012-E03,TMH577-hIgG1-013-H3_H03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
2,TMH577-hF-012-E03,TMH577-hIgG1-013-H7_H07,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
3,TMH577-hF-012-E03,TMH577-hIgG1-014-A5_A05,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
4,TMH577-hF-012-E03,TMH577-hIgG1-014-C8_C08,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...


## Note
- Save a copy of df_vl, df_vh, res_df_copy

### sanity check

In [26]:
df_vh.shape,res_df_copy.shape

((183, 14), (366, 4))

In [27]:
res_df_copy.h3_name.value_counts()

TMH577-hF-014-B12    22
TMH577-hF-015-C08    22
TMH577-hF-015-D08    22
TMH577-hF-017-G04    22
TMH577-hF-012-E03    14
                     ..
TMH577-hF-016-D06     2
TMH577-hF-012-C01     2
TMH577-hF-016-C03     2
TMH577-hF-004-G09     2
TMH577-hF-017-D03     2
Name: h3_name, Length: 67, dtype: int64

In [28]:
sample_ids[0:5]

['TMH577-hIgG1-013-A10',
 'TMH577-hIgG1-013-A11',
 'TMH577-hIgG1-013-A12',
 'TMH577-hIgG1-013-A1_A01',
 'TMH577-hIgG1-013-A2_A02']

In [29]:
# res_df_copy[res_df_copy.sample_id == 'TMH577-hIgG1-013-A10']

# res_df_copy[res_df_copy.sample_id == 'TMH577-hIgG1-013-A10'].abi_out_loc.tolist()

# res_df_copy[res_df_copy.sample_id == 'TMH577-hIgG1-013-H7_H07'].iloc[0]["abi_initial_filepath"]

# res_df_copy[res_df_copy.sample_id == 'TMH577-hIgG1-013-H7_H07'].iloc[1]["abi_initial_filepath"]

In [30]:
# for template_strand in os.listdir(vh_template_sequence_folder):
#     parsed_template_strand = template_strand.split('VH-')[1].split('.gb')[0]
#     print(f"{template_strand} | {parsed_template_strand}")

In [31]:
# df_vh.head(3)

# df_vh[df_vh.h3_name == 'TMH577-hF-004-G09']

## PAIRWISE ALIGNMENT OF GENBANK FILES WITH MATCHED ABI FILES TO GET THE MATCHING SCORE 
- Iterate through the VH (and VL) genbank file names [the file names are in a dict vh_template_gb_dict]
    - if the gb file name matches with h3_name column in df_vh or df_vl 
      - then save that into a list gb_match_with_dfvx_lst
          - GB_Matched|gb_id|h3_name|sample_id|gb_fp|vh_abi_fp|vl_abi_fp
- gb file name could match an h3_name; also this h3_name might be associated with different sample ids
```
[+] GENBANK_MATCHING:|genbank_id|h3_name|sample_id
[+] GENBANK_MATCHING:|VH-TMH577-hF-005-G08|TMH577-hF-005-G08|TMH577-hIgG1-014-A8_A08
[+] GENBANK_MATCHING:|VH-TMH577-hF-005-G08|TMH577-hF-005-G08|TMH577-hIgG1-014-C4_C04
```

In [32]:
# # This loop iterate through the genbank file and try matching the gb sample name with h3_name in the df_vh ( which is matched against the prob) 

# gb_match_with_dfvh_lst = []
# for gb_id in [*vh_template_gb_dict.keys()]:
#     _ = df_vh[df_vh.h3_name == gb_id.replace('VH-', '')]
#     if not (_).empty:
#         for j in range(_.shape[0]):
#             # print(f"Matched | {gb_id} | {_.iloc[j].h3_name} | {_.iloc[j].sample_id} | {vh_template_gb_dict.get(gb_id)}  | {_.iloc[j].vh_abi_fp} | {_.iloc[j].vl_abi_fp}")
#             gb_match_with_dfvh_lst.append(['GB_Matched',gb_id,_.iloc[j].h3_name, _.iloc[j].sample_id, vh_template_gb_dict.get(gb_id), _.iloc[j].vh_abi_fp, _.iloc[j].vl_abi_fp])
#             template = SeqIO.read(vh_template_gb_dict.get(gb_id), 'gb')
#             for a in pairwise2.align.globalms(template.seq, _.iloc[j].abi_seq, 2, -1000, -1000, -1000, penalize_end_gaps = False):
#                 score = int(a[2])
#                 print(f"{gb_id}|{_.iloc[j].h3_name}|{score}")
#     else:
#         pass
#         # print(f"Not Matched {gb_id}")
        
# # gb_h3_matched_df = pd.DataFrame(gb_match_with_dfvh_lst)
# # gb_h3_matched_df.columns = ["Match", "gb_id", "h3_name", "sample_id", "gb_fp", "vh_abi_fp", "vl_abi_fp"]

In [33]:
# def find_gb_match_on_all_h3probes2(gb_tmplate_dict: dict, df_vx: pd.DataFrame, pattern='VH-') -> pd.DataFrame:
#     """
#     """
#     gb_match_with_dfvx_lst = []
#     for gb_id in [*gb_tmplate_dict.keys()]:
#         _ = df_vx[df_vx.h3_name == gb_id.replace(pattern, '')]
#         if not (_).empty:
#             for j in range(_.shape[0]):
#                 # print(f"Matched|{gb_id}|{_.iloc[j].h3_name}|{_.iloc[j].sample_id}|{gb_tmplate_dict.get(gb_id)}|{_.iloc[j].vh_abi_fp}|{_.iloc[j].vl_abi_fp}")
#                 gb_match_with_dfvx_lst.append(['GB_Matched',gb_id, _.iloc[j].h3_name, _.iloc[j].sample_id, gb_tmplate_dict.get(gb_id), _.iloc[j].vh_abi_fp, _.iloc[j].vl_abi_fp])
#             # print()
#         else:
#             pass
#             # print(f"Not Matched {gb_id}")
#     _df = pd.DataFrame(gb_match_with_dfvx_lst)
#     _df.columns = ["Match", "gb_id", "h3_name", "sample_id", "gb_fp", "vh_abi_fp", "vl_abi_fp"]
#     return _df

In [34]:
df_vh.head(2)

,Match,h3_name,sample_id,vh_abi_fp,vl_abi_fp,probe_seq,vh_init_sr,vl_inti_sr,vh_sr_seq_r,vh_sr_trimmed,vh_sr_tqlst,vl_sr_seq_r,vl_sr_trimmed,vl_sr_tqlst
0,Reverse_Strand_Match,TMH577-hF-012-E03,TMH577-hIgG1-013-A10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGACCCTTTTGGAGCGGCCTAACAAGAGAAAACTACTATTACG...,"(A, T, G, C, C, A, A, A, G, C, C, C, A, A, G, ...","(C, G, G, C, A, T, T, C, T, T, G, A, G, G, C, ...","(T, C, A, C, G, G, G, G, A, T, T, T, C, C, A, ...","(G, G, G, C, C, T, G, C, C, C, C, A, G, A, A, ...","[54, 7, 6, 12, 16, 16, 37, 35, 41, 39, 19, 24,...","(A, A, T, A, T, A, T, A, G, C, A, G, T, T, T, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 18, 20, 53, 53, 42, 36, 25, 30, 52, 58, 5..."
1,Reverse_Strand_Match,TMH577-hF-015-C12,TMH577-hIgG1-013-A11,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCTAGGTACTACTACGGTATGGACGTC,"(C, C, G, T, T, C, C, A, A, A, A, C, C, A, A, ...","(G, G, G, A, A, G, G, C, T, G, A, G, C, A, G, ...","(T, T, A, T, G, G, G, A, C, T, T, T, C, C, T, ...","(T, A, C, T, C, A, C, C, T, G, A, G, C, T, C, ...","[58, 47, 37, 58, 31, 24, 54, 58, 58, 58, 45, 5...","(C, G, C, C, A, A, T, C, A, G, A, A, C, C, C, ...","(A, C, A, G, G, G, A, G, G, G, C, A, G, A, G, ...","[54, 12, 9, 32, 22, 13, 6, 12, 58, 24, 22, 47,..."


In [35]:
# def find_gb_match_on_all_h3probes_v1(log: logging, gb_tmplate_dict: dict, df_vx: pd.DataFrame, pattern='VH-') -> pd.DataFrame:
#     """
#     """
#     gb_match_with_dfvx_lst = []
#     print(f"[+] GENBANK_MATCHING:|counter|enum|genbank_id|h3_name|sample_id|gb_fp|VH_abi_fp|VL_abi_fp")
#     c = 0
#     for e,gb_id in enumerate([*gb_tmplate_dict.keys()], start=1):
#         _ = df_vx[df_vx.h3_name == gb_id.replace(pattern, '')]
#         if not (_).empty:
#             for j in range(_.shape[0]):
#                 print(f"[+] GENBANK_MATCHING:|{c}|{e}|{gb_id}|{_.iloc[j].h3_name}|{_.iloc[j].sample_id}|{gb_tmplate_dict.get(gb_id)}|{_.iloc[j].vh_abi_fp}|{_.iloc[j].vl_abi_fp}")
#                 c+=1
#                 # log.info(f"[+] GENBANK_MATCHING: {gb_id}|{_.iloc[j].h3_name}|{_.iloc[j].sample_id}|{gb_tmplate_dict.get(gb_id)}|{_.iloc[j].vh_abi_fp}|{_.iloc[j].vl_abi_fp}")
#                 gb_match_with_dfvx_lst.append(['GB_Matched',gb_id, _.iloc[j].h3_name, _.iloc[j].sample_id, gb_tmplate_dict.get(gb_id), _.iloc[j].vh_abi_fp, _.iloc[j].vl_abi_fp])
#             # print()
#         else:
#             pass
#             # print(f"Not Matched {gb_id}")
#     _df = pd.DataFrame(gb_match_with_dfvx_lst)
#     _df.columns = ["Match", "gb_id", "h3_name", "sample_id", "gb_fp", "vh_abi_fp", "vl_abi_fp"]
#     return _df

In [36]:
def find_gb_match_on_all_h3probes_v2(log: logging, gb_tmplate_dict: dict, df_vx: pd.DataFrame, pattern='VH-', log_msg: bool=True) -> pd.DataFrame:
    """
    """
    gb_match_with_dfvx_lst = []
    for gb_id in [*gb_tmplate_dict.keys()]:
        _ = df_vx[df_vx.h3_name == gb_id.replace(pattern, '')]
        if not (_).empty:
            for j in range(_.shape[0]):
                log.info(f"[+] GENBANK_MATCHING: {gb_id}|{_.iloc[j].h3_name}|{_.iloc[j].sample_id}|{gb_tmplate_dict.get(gb_id)}|{_.iloc[j].vh_abi_fp}|{_.iloc[j].vl_abi_fp}") if log_msg else None
                gb_match_with_dfvx_lst.append(['GB_Matched',gb_id, _.iloc[j].h3_name, _.iloc[j].sample_id, gb_tmplate_dict.get(gb_id), _.iloc[j].vh_abi_fp, _.iloc[j].vl_abi_fp])
        else:
            pass
    _df = pd.DataFrame(gb_match_with_dfvx_lst)
    _df.columns = ["Match", "gb_id", "h3_name", "sample_id", "gb_fp", "vh_abi_fp", "vl_abi_fp"]
    return _df

In [37]:
# M_gb_abi_vh2 = find_gb_match_on_all_h3probes(log, vh_template_gb_dict, df_vl, pattern=patrm_vh_gb)
# M_gb_abi_vl = find_gb_match_on_all_h3probes(log, vl_template_gb_dict, df_vh, pattern=patrm_vl_gb)
# M_gb_abi_vl2 = find_gb_match_on_all_h3probes(log, vl_template_gb_dict, df_vl, pattern=patrm_vl_gb)

In [38]:
log.info(f"\nFIND THE GB FILENASMES WITH H3 NAMES")
M_gb_abi_vh = find_gb_match_on_all_h3probes_v2(log, vh_template_gb_dict, df_vh, pattern=patrm_vh_gb, log_msg=True)
M_gb_abi_vl = find_gb_match_on_all_h3probes_v2(log, vl_template_gb_dict, df_vh, pattern=patrm_vl_gb, log_msg=True)

In [39]:
all_gb_match = pd.concat([M_gb_abi_vh, M_gb_abi_vl]).reset_index(drop=True)

In [40]:
all_gb_match.shape

(366, 7)

In [41]:
all_gb_match[['gb_id', 'h3_name', 'sample_id']].drop_duplicates()

,gb_id,h3_name,sample_id
0,VH-TMH577-hF-004-G09,TMH577-hF-004-G09,TMH577-hIgG1-013-A4_A04
1,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-A8_A08
2,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-C4_C04
3,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-013-F12
4,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-014-G9_G09
...,...,...,...
361,VL-TMH577-hF-017-H02,TMH577-hF-017-H02,TMH577-hIgG1-013-B12
362,VL-TMH577-hF-017-H02,TMH577-hF-017-H02,TMH577-hIgG1-014-H4_H04
363,VL-TMH577-hF-017-H03,TMH577-hF-017-H03,TMH577-hIgG1-013-A2_A02
364,VL-TMH577-hF-017-H03,TMH577-hF-017-H03,TMH577-hIgG1-013-C10


In [42]:
all_gb_match

,Match,gb_id,h3_name,sample_id,gb_fp,vh_abi_fp,vl_abi_fp
0,GB_Matched,VH-TMH577-hF-004-G09,TMH577-hF-004-G09,TMH577-hIgG1-013-A4_A04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
1,GB_Matched,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-A8_A08,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
2,GB_Matched,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-C4_C04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
3,GB_Matched,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-013-F12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
4,GB_Matched,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-014-G9_G09,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
...,...,...,...,...,...,...,...
361,GB_Matched,VL-TMH577-hF-017-H02,TMH577-hF-017-H02,TMH577-hIgG1-013-B12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
362,GB_Matched,VL-TMH577-hF-017-H02,TMH577-hF-017-H02,TMH577-hIgG1-014-H4_H04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
363,GB_Matched,VL-TMH577-hF-017-H03,TMH577-hF-017-H03,TMH577-hIgG1-013-A2_A02,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
364,GB_Matched,VL-TMH577-hF-017-H03,TMH577-hF-017-H03,TMH577-hIgG1-013-C10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...


In [43]:
M_gb_abi_vh

,Match,gb_id,h3_name,sample_id,gb_fp,vh_abi_fp,vl_abi_fp
0,GB_Matched,VH-TMH577-hF-004-G09,TMH577-hF-004-G09,TMH577-hIgG1-013-A4_A04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
1,GB_Matched,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-A8_A08,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
2,GB_Matched,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-C4_C04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
3,GB_Matched,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-013-F12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
4,GB_Matched,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-014-G9_G09,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
...,...,...,...,...,...,...,...
178,GB_Matched,VH-TMH577-hF-017-H02,TMH577-hF-017-H02,TMH577-hIgG1-013-B12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
179,GB_Matched,VH-TMH577-hF-017-H02,TMH577-hF-017-H02,TMH577-hIgG1-014-H4_H04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
180,GB_Matched,VH-TMH577-hF-017-H03,TMH577-hF-017-H03,TMH577-hIgG1-013-A2_A02,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
181,GB_Matched,VH-TMH577-hF-017-H03,TMH577-hF-017-H03,TMH577-hIgG1-013-C10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...


In [44]:
M_gb_abi_vl

,Match,gb_id,h3_name,sample_id,gb_fp,vh_abi_fp,vl_abi_fp
0,GB_Matched,VL-TMH577-hF-004-G09,TMH577-hF-004-G09,TMH577-hIgG1-013-A4_A04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
1,GB_Matched,VL-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-A8_A08,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
2,GB_Matched,VL-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-C4_C04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
3,GB_Matched,VL-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-013-F12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
4,GB_Matched,VL-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-014-G9_G09,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
...,...,...,...,...,...,...,...
178,GB_Matched,VL-TMH577-hF-017-H02,TMH577-hF-017-H02,TMH577-hIgG1-013-B12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
179,GB_Matched,VL-TMH577-hF-017-H02,TMH577-hF-017-H02,TMH577-hIgG1-014-H4_H04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
180,GB_Matched,VL-TMH577-hF-017-H03,TMH577-hF-017-H03,TMH577-hIgG1-013-A2_A02,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
181,GB_Matched,VL-TMH577-hF-017-H03,TMH577-hF-017-H03,TMH577-hIgG1-013-C10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...


In [45]:
vh_template_gb_dict.get(all_gb_match.iloc[0].gb_id)

'/Users/rp/Desktop/purge/ny-immuno-sanger/data02/Template/VH/VH-TMH577-hF-004-G09.gb'

In [46]:
res_df_copy[res_df_copy.h3_name == all_gb_match.iloc[1].h3_name]['abi_initial_filepath'].tolist()

['/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-014-A8_A08_GATC-VH60-2617917.abi',
 '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-014-C4_C04_GATC-VH60-2617917.abi',
 '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-014-A8_A08_VL79.abi',
 '/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-014-C4_C04_VL79.abi']

In [47]:
# find_gb_match_on_all_h3probes(vl_template_gb_dict, df_vh, pattern='VL-')

#### Sanity check

In [48]:
len(vh_abi_dict),len(vl_abi_dict), len(sample_ids)

(186, 186, 186)

In [49]:
 df_vh.shape

(183, 14)

In [50]:
len(vh_template_gb_dict), len(vl_template_gb_dict)

(1917, 1935)

In [51]:
# M_gb_abi_vh.shape ,M_gb_abi_vh2.shape, M_gb_abi_vl.shape, M_gb_abi_vl2.shape

In [52]:
# match = 2
# mismatch = -1000
# open = -1000
# extend = -1000
# filtering = 700
# penalize_end_gaps = False

In [53]:
def align(A_seq: SeqIO.SeqRecord, B_seq: SeqIO.SeqRecord, match :int, mismatch: int, gap_open: int, gap_extend: int):
    return pairwise2.align.globalms(sequenceA=A_seq, sequenceB=B_seq, match=match, mismatch=mismatch, open=gap_open, extend=gap_extend, penalize_end_gaps = False)

In [54]:
# for i in align(template.seq, template.seq, match=1, mismatch=-10, gap_open=-10, gap_extend=-10):
#     print(format_alignment(*i))

In [55]:
# all_abi_probe_match.head()

In [56]:
# for each genbank file that matched with probe name (h3_name)
# align the abi files for VH and then VL 
# collect the alignment scores 

In [57]:
# all_abi_probe_match

In [58]:
# print(f"outer_index | inner_index | VH | GBMacthVH | H3_name | GB_FP | ABI_VH_FP | Score | Quality_score | Low_quality ")
# for e,i in enumerate(range(all_gb_match.shape[0])[:]):
#     gbid = all_gb_match.iloc[i].gb_id
#     gbfp = all_gb_match.iloc[i].gb_fp
#     _dfvx_ss = all_abi_probe_match[all_abi_probe_match.h3_name == all_gb_match.iloc[i].h3_name]   
#     for e2, abx in enumerate(range(_dfvx_ss.shape[0])):
#         # match between gb and vh-abi
#         try:
#             seqA = SeqIO.read(vh_template_gb_dict.get(gbid), 'gb')
#             seqB = _dfvx_ss.iloc[abx].abi_seq
#             seqBinit = _dfvx_ss.iloc[abx].seq_record.seq
#             quality = _dfvx_ss.iloc[abx].trimmed_seq_qual
#             for a in align(seqA.seq, seqB, match=int(par_match), mismatch=int(par_missmatch), gap_open=int(par_open), gap_extend=int(par_extend)):
#                 score = int(a[2])
#                 if score >= int(par_filter_thresh):
                    
#                     # getting the aligned seqA and seqB
#                     aligned_seq_a = a[0]
#                     aligned_seq_a = aligned_seq_a.replace('-', '')
#                     aligned_seq_a_start = aligned_seq_a[0:6]
#                     aligned_seq_a_length = len(aligned_seq_a) - 6
#                     aligned_seq_a_end = aligned_seq_a[aligned_seq_a_length:len(aligned_seq_a)]
#                     aligned_seq_b = a[1]
#                     # futher filtering of seqB
#                     if '-' not in aligned_seq_b:
#                         if 'CTCCAC' in aligned_seq_a_start:
#                             if 'TTCTGG' in aligned_seq_a_end: #CTTTCT
#                                 aligned_seq = a[0]
#                                 aligned_seq = aligned_seq.replace('-', '')
#                                 aligned_seq = Seq(aligned_seq)
#                                 aligned_seq_rev = aligned_seq.reverse_complement()
#                                 dna_string = str(seqBinit)
#                                 aligned_seq_rev = str(aligned_seq_rev)
#                                 start = re.search(aligned_seq_rev, dna_string).start()
#                                 end = re.search(aligned_seq_rev, dna_string).end()
#                                 quality = quality[start:end]
#                                 quality_score = statistics.mean(quality)
#                                 lowest_quality = min(quality)
#                                 print(f"{e} | {abx} | VH | {gbid} | {_dfvx_ss.iloc[abx].h3_name} | {gbfp} | {_dfvx_ss.iloc[abx].vh_abi_fp} | {score}| {quality_score} | {lowest_quality}")
#         except:
#             pass          

In [59]:
# # The score is pretty low for these VH

# print(f"outer_index | inner_index | VL | GBMacthVH | H3_name | GB_FP | ABI_VH_FP | Score | Quality_score | Low_quality ")

# # M_gb_abi_vh
# # df_vh
# # vh_template_gb_dict


# for e,i in enumerate(range(M_gb_abi_vh.shape[0])[:]):                                # change 
#     gbid = M_gb_abi_vh.iloc[i].gb_id                                                 # change 
#     gbfp = M_gb_abi_vh.iloc[i].gb_fp                                                 # change 
#     _dfvx_ss = df_vh[df_vh.h3_name == M_gb_abi_vh.iloc[i].h3_name]                   # change 
#     for e2, abx in enumerate(range(_dfvx_ss.shape[0])):
#         # match between gb and vl-abi
#         try:
#             seqA = SeqIO.read(vh_template_gb_dict.get(gbid), 'gb')                   # change 
#             seqB = _dfvx_ss.iloc[abx].abi_seq
#             seqBinit = _dfvx_ss.iloc[abx].seq_record.seq
#             quality = _dfvx_ss.iloc[abx].trimmed_seq_qual
            
#             for a in align(seqA.seq, seqB, match=int(par_match), mismatch=int(par_missmatch), gap_open=int(par_open), gap_extend=int(par_extend)):
#                 score = int(a[2])
#                 # print(f"{e} | {abx} | VL | {gbid} | {score} ")
#                 if score >= int(par_filter_thresh):
#                     # print(f"{e} | {abx} | VL | {gbid} | {score} ") #{aligned_seq_a_start}, {aligned_seq_a_end}")
#                     # getting the aligned seqA and seqB
#                     aligned_seq_a = a[0]
#                     aligned_seq_a = aligned_seq_a.replace('-', '')
#                     aligned_seq_a_start = aligned_seq_a[0:6]
#                     aligned_seq_a_length = len(aligned_seq_a) - 6
#                     aligned_seq_a_end = aligned_seq_a[aligned_seq_a_length:len(aligned_seq_a)]
#                     aligned_seq_b = a[1]
                    
#                     # futher filtering of seqB
#                     if '-' not in aligned_seq_b:
#                         if vh_seq_start_a in aligned_seq_a_start:                        # change 
#                             if vh_seq_end_a in aligned_seq_a_end: #CTTTCT                # change 
#                                 aligned_seq = a[0]
#                                 aligned_seq = aligned_seq.replace('-', '')
#                                 aligned_seq = Seq(aligned_seq)
#                                 aligned_seq_rev = aligned_seq.reverse_complement()
#                                 dna_string = str(seqBinit)
#                                 aligned_seq_rev = str(aligned_seq_rev)
#                                 start = re.search(aligned_seq_rev, dna_string).start()
#                                 end = re.search(aligned_seq_rev, dna_string).end()
#                                 quality = quality[start:end]
#                                 quality_score = statistics.mean(quality)
#                                 lowest_quality = min(quality)
#                                 print(f"{e} | {abx} | VH | {gbid} | {_dfvx_ss.iloc[abx].h3_name} | {gbfp} | {_dfvx_ss.iloc[abx].vh_abi_fp} | {score}| {quality_score} | {lowest_quality}")
#                 else:
#                     print(f"{e} | {abx} | VH | {gbid} | {_dfvx_ss.iloc[abx].h3_name} | {gbfp} | {_dfvx_ss.iloc[abx].vh_abi_fp} | {score}| --- | ---- ")
                    
#         except:
#             pass  

In [60]:
# # The score is pretty low for these VL

# print(f"outer_index | inner_index | VL | GBMacthVH | H3_name | GB_FP | ABI_VH_FP | Score | Quality_score | Low_quality ")

# # M_gb_abi_vl
# # df_vl
# # vl_template_gb_dict


# for e,i in enumerate(range(M_gb_abi_vl.shape[0])[:]):
#     gbid = M_gb_abi_vl.iloc[i].gb_id
#     gbfp = M_gb_abi_vl.iloc[i].gb_fp
#     _dfvx_ss = df_vl[df_vl.h3_name == M_gb_abi_vl.iloc[i].h3_name]   
#     for e2, abx in enumerate(range(_dfvx_ss.shape[0])):
#         # match between gb and vl-abi
#         try:
#             seqA = SeqIO.read(vl_template_gb_dict.get(gbid), 'gb')  # change 
#             seqB = _dfvx_ss.iloc[abx].abi_seq
#             seqBinit = _dfvx_ss.iloc[abx].seq_record.seq
#             quality = _dfvx_ss.iloc[abx].trimmed_seq_qual
            
#             for a in align(seqA.seq, seqB, match=int(par_match), mismatch=int(par_missmatch), gap_open=int(par_open), gap_extend=int(par_extend)):
#                 score = int(a[2])
#                 #print(f"{e} | {abx} | VL | {gbid} | {score} ")
#                 if score >= int(par_filter_thresh):
#                     # print(f"{e} | {abx} | VL | {gbid} | {score} ") #{aligned_seq_a_start}, {aligned_seq_a_end}")
#                     # getting the aligned seqA and seqB
#                     aligned_seq_a = a[0]
#                     aligned_seq_a = aligned_seq_a.replace('-', '')
#                     aligned_seq_a_start = aligned_seq_a[0:6]
#                     aligned_seq_a_length = len(aligned_seq_a) - 6
#                     aligned_seq_a_end = aligned_seq_a[aligned_seq_a_length:len(aligned_seq_a)]
#                     aligned_seq_b = a[1]
                    
#                     # futher filtering of seqB
#                     if '-' not in aligned_seq_b:
#                         if 'CTCCAC' in aligned_seq_a_start:
#                             if 'GCTTGG' in aligned_seq_a_end: #CTTTCT
#                                 aligned_seq = a[0]
#                                 aligned_seq = aligned_seq.replace('-', '')
#                                 aligned_seq = Seq(aligned_seq)
#                                 aligned_seq_rev = aligned_seq.reverse_complement()
#                                 dna_string = str(seqBinit)
#                                 aligned_seq_rev = str(aligned_seq_rev)
#                                 start = re.search(aligned_seq_rev, dna_string).start()
#                                 end = re.search(aligned_seq_rev, dna_string).end()
#                                 quality = quality[start:end]
#                                 quality_score = statistics.mean(quality)
#                                 lowest_quality = min(quality)
#                                 print(f"{e} | {abx} | VH | {gbid} | {_dfvx_ss.iloc[abx].h3_name} | {gbfp} | {_dfvx_ss.iloc[abx].vh_abi_fp} | {score}| {quality_score} | {lowest_quality}")
#                 else:
#                     print(f"{e} | {abx} | VH | {gbid} | {_dfvx_ss.iloc[abx].h3_name} | {gbfp} | {_dfvx_ss.iloc[abx].vh_abi_fp} | {score}| --- | ---- ")
#         except:
#             pass  

In [61]:
# print(f"outer_index | inner_index | VL | GBMacthVH | H3_name | GB_FP | ABI_VL_FP | Score | ")
# for e,i in enumerate(range(all_gb_match.shape[0])):
#     gbid = all_gb_match.iloc[i].gb_id
#     gbfp = all_gb_match.iloc[i].gb_fp
#     _dfvx_ss = df_vl[df_vl.h3_name == all_gb_match.iloc[i].h3_name]
#     # print(_dfvx_ss.Match)
#     for e2, abx in enumerate(range(_dfvx_ss.shape[0])):
#         # match between gb and vl-abi
#         try:
#             seqA = SeqIO.read(vl_template_gb_dict.get(gbid), 'gb')
#             x, seqB, xx = get_seqobj_from_abi(_dfvx_ss.iloc[abx].vl_abi_fp)
#             seqB_rseq = get_seq_from_record(seqB, reverse=True)

#             for a in align(seqA.seq, seqB_rseq, match=int(par_match), mismatch=int(par_missmatch), gap_open=int(par_open), gap_extend=int(par_extend)):
#                 score = int(a[2])
#                 print(f"{e} | {abx} | VL | {gbid} | {_dfvx_ss.iloc[abx].h3_name} | {gbfp} | {_dfvx_ss.iloc[abx].vl_abi_fp} | {score}")
#                 if score >= int(par_filter_thresh):
#                     print(f"{e} | {abx} | VL | {gbid} | {_dfvx_ss.iloc[abx].h3_name} | {gbfp} | {_dfvx_ss.iloc[abx].vl_abi_fp} | {score}")
#         except :
#             #print(f"{e}|{gbid} | {_dfvx_ss.iloc[abx].h3_name}")
#             pass
        

In [61]:
if score >= 700:
    print(f"Aseq  : {a[0]}")
    print(f"Bseq  : {a[1]}")
    print(f"Score : {a[2]}")
    print(f"Start : {a[3]}")
    print(f"End   : {a[4]}")
    aligned_seq_a = a[0]
    aligned_seq_a = aligned_seq_a.replace('-', '')
    print(f"aligned_seq_a : {aligned_seq_a}")
    aligned_seq_a_start = aligned_seq_a[0:6]
    print(f"aligned_seq_a_start : {aligned_seq_a_start}")
    aligned_seq_a_length = len(aligned_seq_a) - 6
    print(f"a_seq_len : {len(a[0])} | aligned_seq_a_length: {aligned_seq_a_length}")
    aligned_seq_a_end = aligned_seq_a[aligned_seq_a_length:len(aligned_seq_a)]
    print(f"aligned_seq_a_end : {aligned_seq_a_end}")
    
    aligned_seq_b = a[1]
    print(f"aligned_seq_b : {aligned_seq_b}")

    #check for gaps in template during alignment
    if '-' not in aligned_seq_b:
        #check aligned seq starts with CTCCAC
        if 'CTCCAC' in aligned_seq_a_start:
            #check aligned seq ends with GCTTGG
            if 'GCTTGG' in aligned_seq_a_end:
                print("Im here")
                count += 1
                print(filename)
                print(template_strand)
                print('\n')
                print(format_alignment(*a))

                print('\n')

                aligned_seq = a[0]
                aligned_seq = aligned_seq.replace('-', '')
                aligned_seq = Seq(aligned_seq)
                aligned_seq_rev = aligned_seq.reverse_complement()
                dna_string = str(dna)
                aligned_seq_rev = str(aligned_seq_rev)
                start = re.search(aligned_seq_rev, dna_string).start()
                end = re.search(aligned_seq_rev, dna_string).end()

                quality = quality[start:end]
                quality_score = statistics.mean(quality)
                lowest_quality = min(quality)

Aseq  : -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CTCCACAGGTGTGCATTCCGAGGTGCAGCTGTTGGAGTCTGGGGGAGGCTTGGTACAGCCTGGGGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTTAATAACTATCCCATGACCTGGGTCCGCCGGGCTCCAGGGAAGGGGCTGGAGTGGGTCTCAGCTATCAGTGGTAGTGGTAGAAGCACATACCACGCAGACTCCGTGAAGGGCCGGTTCACCATCTCCAGAGACAATTCCAAGAACACGCTGTATCTGCAAATGAACAGCCTGAGAGCCGAGGACACTGCCGTGTATTACTGTGCGAAAGGAGACCG

In [62]:
M_gb_abi_vh.head()

,Match,gb_id,h3_name,sample_id,gb_fp,vh_abi_fp,vl_abi_fp
0,GB_Matched,VH-TMH577-hF-004-G09,TMH577-hF-004-G09,TMH577-hIgG1-013-A4_A04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
1,GB_Matched,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-A8_A08,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
2,GB_Matched,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-C4_C04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
3,GB_Matched,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-013-F12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
4,GB_Matched,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-014-G9_G09,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...


In [63]:
df_vh.head()

,Match,h3_name,sample_id,vh_abi_fp,vl_abi_fp,probe_seq,vh_init_sr,vl_inti_sr,vh_sr_seq_r,vh_sr_trimmed,vh_sr_tqlst,vl_sr_seq_r,vl_sr_trimmed,vl_sr_tqlst
0,Reverse_Strand_Match,TMH577-hF-012-E03,TMH577-hIgG1-013-A10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGACCCTTTTGGAGCGGCCTAACAAGAGAAAACTACTATTACG...,"(A, T, G, C, C, A, A, A, G, C, C, C, A, A, G, ...","(C, G, G, C, A, T, T, C, T, T, G, A, G, G, C, ...","(T, C, A, C, G, G, G, G, A, T, T, T, C, C, A, ...","(G, G, G, C, C, T, G, C, C, C, C, A, G, A, A, ...","[54, 7, 6, 12, 16, 16, 37, 35, 41, 39, 19, 24,...","(A, A, T, A, T, A, T, A, G, C, A, G, T, T, T, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 18, 20, 53, 53, 42, 36, 25, 30, 52, 58, 5..."
1,Reverse_Strand_Match,TMH577-hF-015-C12,TMH577-hIgG1-013-A11,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCTAGGTACTACTACGGTATGGACGTC,"(C, C, G, T, T, C, C, A, A, A, A, C, C, A, A, ...","(G, G, G, A, A, G, G, C, T, G, A, G, C, A, G, ...","(T, T, A, T, G, G, G, A, C, T, T, T, C, C, T, ...","(T, A, C, T, C, A, C, C, T, G, A, G, C, T, C, ...","[58, 47, 37, 58, 31, 24, 54, 58, 58, 58, 45, 5...","(C, G, C, C, A, A, T, C, A, G, A, A, C, C, C, ...","(A, C, A, G, G, G, A, G, G, G, C, A, G, A, G, ...","[54, 12, 9, 32, 22, 13, 6, 12, 58, 24, 22, 47,..."
2,Reverse_Strand_Match,TMH577-hF-014-B12,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ...","(G, T, G, G, A, A, C, G, T, C, C, T, G, A, G, ...","(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, T, G, G, C, C, C, C, A, G, A, C, G, T, C, ...","[50, 56, 59, 43, 39, 56, 56, 43, 36, 34, 49, 3...","(A, C, G, C, A, G, C, A, C, A, A, A, C, G, A, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 18, 20, 50, 51, 32, 32, 25, 30, 51, 58, 5..."
3,Reverse_Strand_Match,TMH577-hF-015-C08,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ...","(G, T, G, G, A, A, C, G, T, C, C, T, G, A, G, ...","(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, T, G, G, C, C, C, C, A, G, A, C, G, T, C, ...","[50, 56, 59, 43, 39, 56, 56, 43, 36, 34, 49, 3...","(A, C, G, C, A, G, C, A, C, A, A, A, C, G, A, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 18, 20, 50, 51, 32, 32, 25, 30, 51, 58, 5..."
4,Reverse_Strand_Match,TMH577-hF-015-D08,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ...","(G, T, G, G, A, A, C, G, T, C, C, T, G, A, G, ...","(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, T, G, G, C, C, C, C, A, G, A, C, G, T, C, ...","[50, 56, 59, 43, 39, 56, 56, 43, 36, 34, 49, 3...","(A, C, G, C, A, G, C, A, C, A, A, A, C, G, A, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 18, 20, 50, 51, 32, 32, 25, 30, 51, 58, 5..."


In [64]:
df_vh.columns

Index(['Match', 'h3_name', 'sample_id', 'vh_abi_fp', 'vl_abi_fp', 'probe_seq',
       'vh_init_sr', 'vl_inti_sr', 'vh_sr_seq_r', 'vh_sr_trimmed',
       'vh_sr_tqlst', 'vl_sr_seq_r', 'vl_sr_trimmed', 'vl_sr_tqlst'],
      dtype='object')

In [135]:
M_gb_abi_vx = M_gb_abi_vh
df_vx = df_vh
vx_template_gb_dict = vh_template_gb_dict
log_msg = True

filtered_res = []
print(f"[+] FILTERED_MATCH: outer_index|inner_index|GB_ID_MATCHED|H3_name|GB_FP|ABI_VH_FP|Score|Quality_score|Low_quality")if log_msg else None
for e,i in enumerate(range(M_gb_abi_vx.shape[0])[0:]):
    gbid = M_gb_abi_vx.iloc[i].gb_id
    gbfp = M_gb_abi_vx.iloc[i].gb_fp
    _dfvx_ss = df_vx[df_vx.h3_name == M_gb_abi_vx.iloc[i].h3_name]   
    for e2, abx in enumerate(range(_dfvx_ss.shape[0])):
        
        # genbank
        seqA = SeqIO.read(vx_template_gb_dict.get(gbid), 'gb')
        
        # vh
        seqB_vh = _dfvx_ss.iloc[abx].vh_sr_seq_r
        seqB_vhinit = _dfvx_ss.iloc[abx].vh_init_sr.seq
        quality_vh = _dfvx_ss.iloc[abx].vh_sr_tqlst
        
        # vl
        seqB_vl = _dfvx_ss.iloc[abx].vl_sr_seq_r
        seqB_vlinit = _dfvx_ss.iloc[abx]["vl_inti_sr"].seq
        quality_vl = _dfvx_ss.iloc[abx].vl_sr_tqlst  
        
        # match between gb and vh-abi
        try:          
                        
            for a in align(seqA.seq, seqB_vh, match=par_match, mismatch=par_missmatch, gap_open=par_open, gap_extend=par_extend):
                score = int(a[2])
                if score >= int(par_filter_thresh):
                    
                    # getting the aligned seqA and seqB
                    aligned_seq_a = a[0]
                    aligned_seq_a = aligned_seq_a.replace('-', '')
                    aligned_seq_a_start = aligned_seq_a[0:6]
                    aligned_seq_a_length = len(aligned_seq_a) - 6
                    aligned_seq_a_end = aligned_seq_a[aligned_seq_a_length:len(aligned_seq_a)]
                    aligned_seq_b = a[1]
                    # futher filtering of seqB
                    if '-' not in aligned_seq_b:
                        if vh_seq_start_a in aligned_seq_a_start:
                            if vh_seq_end_a in aligned_seq_a_end: 
                                
                                aligned_seq = a[0]
                                aligned_seq = aligned_seq.replace('-', '')
                                
                                aligned_seq = Seq(aligned_seq)
                                aligned_seq_rev = aligned_seq.reverse_complement()
                                
                                dna_string = str(seqB_vhinit)
                                
                                aligned_seq_rev = str(aligned_seq_rev)
                                start = re.search(aligned_seq_rev, dna_string).start()
                                end = re.search(aligned_seq_rev, dna_string).end()
                                
                                # print(f"{e} | {abx} | VH | {score} | {aligned_seq_a_start} | {start} | {end} ")
                                
                                
                                quality_vh = quality_vh[start:end]
                                quality_score = statistics.mean(quality_vh)
                                lowest_quality = min(quality_vh)
                                
                                # print(f"{e} | {abx} | {gbid} | {_dfvx_ss.iloc[abx].h3_name} | {gbfp} | {_dfvx_ss.iloc[abx].vh_abi_fp} | {score}| {quality_score} | {lowest_quality}")
                                print(f"[+] FILTERED_GOODMATCH: {e}|{abx}|VH|{gbid}|{_dfvx_ss.iloc[abx].h3_name}|{gbfp}|{_dfvx_ss.iloc[abx].vh_abi_fp}|{score}|{quality_score}|{lowest_quality}")if log_msg else None
                                filtered_res.append([gbid, _dfvx_ss.iloc[abx].h3_name, gbfp, _dfvx_ss.iloc[abx].vh_abi_fp, score, quality_score, lowest_quality])
            else:
                log.info(f"[+] FILTERED_BADMATCH: {e}|{abx}|VH|{gbid}|{_dfvx_ss.iloc[abx].h3_name}|{gbfp}|{_dfvx_ss.iloc[abx].vh_abi_fp}|{score}|---|---")if log_msg else None
        except:
            pass
# if not len(filtered_res) == 0:
#     dfres = pd.DataFrame(filtered_res)
#     dfres.columns = ["gbid", "H3_name", "GB_FP", "ABI_FP", "Score", "Quality_score", "Low_quality"]
# else:
#     None

[+] FILTERED_MATCH: outer_index|inner_index|GB_ID_MATCHED|H3_name|GB_FP|ABI_VH_FP|Score|Quality_score|Low_quality
[+] FILTERED_GOODMATCH: 0|0|VH|VH-TMH577-hF-004-G09|TMH577-hF-004-G09|/Users/rp/Desktop/purge/ny-immuno-sanger/data02/Template/VH/VH-TMH577-hF-004-G09.gb|/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A4_A04_GATC-VH60-2617917.abi|782|60.79283887468031|43
[+] FILTERED_GOODMATCH: 3|0|VH|VH-TMH577-hF-012-A02|TMH577-hF-012-A02|/Users/rp/Desktop/purge/ny-immuno-sanger/data02/Template/VH/VH-TMH577-hF-012-A02.gb|/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-F12_GATC-VH60-2617917.abi|842|61.11401425178147|43
[+] FILTERED_GOODMATCH: 4|0|VH|VH-TMH577-hF-012-A02|TMH577-hF-012-A02|/Users/rp/Desktop/purge/ny-immuno-sanger/data02/Template/VH/VH-TMH577-hF-012-A02.gb|/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-F12_GATC-VH60-2617917.abi|842|61.11401425178147|43
[+] FILTERED_GOODMATCH: 6|1|VH|VH-TMH577-hF-01

In [75]:
# _dfvx_ss

,Match,h3_name,sample_id,vh_abi_fp,vl_abi_fp,probe_seq,vh_init_sr,vl_inti_sr,vh_sr_seq_r,vh_sr_trimmed,vh_sr_tqlst,vl_sr_seq_r,vl_sr_trimmed,vl_sr_tqlst
62,Reverse_Strand_Match,TMH577-hF-012-A02,TMH577-hIgG1-013-F12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGAGGGCCGCATAGCAGTGGCTGGTCACTCAATTACTACTACT...,"(C, C, G, C, T, C, C, A, A, G, G, C, C, A, A, ...","(G, T, G, G, A, A, A, G, G, C, T, T, A, G, G, ...","(A, T, G, A, C, C, G, G, T, A, A, A, A, T, G, ...","(T, T, A, C, T, C, A, C, C, T, G, A, G, C, T, ...","[58, 54, 46, 37, 58, 31, 18, 58, 58, 58, 58, 3...","(T, T, T, C, T, T, A, T, G, T, T, C, T, T, A, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 25, 20, 50, 50, 48, 42, 36, 35, 54, 58, 5..."
177,Reverse_Strand_Match,TMH577-hF-012-A02,TMH577-hIgG1-014-G9_G09,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGAGGGCCGCATAGCAGTGGCTGGTCACTCAATTACTACTACT...,"(C, G, G, G, T, C, A, A, A, A, G, G, C, C, A, ...","(G, G, G, A, A, G, T, C, T, G, A, A, G, C, A, ...","(T, C, C, T, C, C, A, T, A, T, T, T, C, T, G, ...","(C, T, G, C, C, C, C, A, G, A, A, A, G, C, T, ...","[51, 42, 47, 24, 15, 10, 22, 18, 53, 48, 58, 4...","(C, G, T, G, C, T, G, A, A, G, G, A, G, A, T, ...","(A, C, A, G, G, G, A, G, G, G, C, A, G, A, G, ...","[54, 13, 11, 58, 24, 19, 10, 28, 58, 22, 22, 4..."


In [85]:
# _dfvx_ss['vh_init_sr'].apply(lambda x: len(x))
# _dfvx_ss['vl_inti_sr'].apply(lambda x: len(x))

In [86]:
_dfvx_ss['vl_inti_sr']

62     (G, T, G, G, A, A, A, G, G, C, T, T, A, G, G, ...
177    (G, G, G, A, A, G, T, C, T, G, A, A, G, C, A, ...
Name: vl_inti_sr, dtype: object

In [ ]:
# Note : 
#     somehow VL seqs when aligning to gb file doesnt give good scores

In [105]:
M_gb_abi_vx = M_gb_abi_vh
df_vx = df_vh
vx_template_gb_dict = vh_template_gb_dict
log_msg = True

filtered_res = []
print(f"[+] FILTERED_MATCH: outer_index|inner_index|GB_ID_MATCHED|H3_name|GB_FP|ABI_VH_FP|Score|Quality_score|Low_quality")if log_msg else None
for e,i in enumerate(range(M_gb_abi_vx.shape[0])[0:]):
    gbid = M_gb_abi_vx.iloc[i].gb_id
    gbfp = M_gb_abi_vx.iloc[i].gb_fp
    _dfvx_ss = df_vx[df_vx.h3_name == M_gb_abi_vx.iloc[i].h3_name]   
    for e2, abx in enumerate(range(_dfvx_ss.shape[0])):
        
        # genbank
        seqA = SeqIO.read(vx_template_gb_dict.get(gbid), 'gb')
        
        # vh
        # seqB_vh = _dfvx_ss.iloc[abx].vh_sr_seq_r
        # seqB_vhinit = _dfvx_ss.iloc[abx].vh_init_sr.seq
        # quality_vh = _dfvx_ss.iloc[abx].vh_sr_tqlst
        
        # vl
        
        seqB = get_seqobj_from_abi(_dfvx_ss.iloc[abx].vl_abi_fp)
        seqB_vl, quality_vl = get_trimmed_seq_record(seqB, get_quality=True)
        seqB_vl = get_seq_from_record(seqB_vl, reverse=True)
        
        # seqB_vl = _dfvx_ss.iloc[abx].vl_sr_seq_r
        seqB_vlinit = _dfvx_ss.iloc[abx]["vl_inti_sr"].seq
        # quality_vl = _dfvx_ss.iloc[abx].vl_sr_tqlst  
        
        # match between gb and vh-abi
#         try:          
                        
#             for a in align(seqA.seq, seqB_vh, match=par_match, mismatch=par_missmatch, gap_open=par_open, gap_extend=par_extend):
#                 score = int(a[2])
#                 if score >= int(par_filter_thresh):
                    
#                     # getting the aligned seqA and seqB
#                     aligned_seq_a = a[0]
#                     aligned_seq_a = aligned_seq_a.replace('-', '')
#                     aligned_seq_a_start = aligned_seq_a[0:6]
#                     aligned_seq_a_length = len(aligned_seq_a) - 6
#                     aligned_seq_a_end = aligned_seq_a[aligned_seq_a_length:len(aligned_seq_a)]
#                     aligned_seq_b = a[1]
#                     # futher filtering of seqB
#                     if '-' not in aligned_seq_b:
#                         if vh_seq_start_a in aligned_seq_a_start:
#                             if vh_seq_end_a in aligned_seq_a_end: 
#                                 aligned_seq = a[0]
#                                 aligned_seq = aligned_seq.replace('-', '')
#                                 aligned_seq = Seq(aligned_seq)
#                                 aligned_seq_rev = aligned_seq.reverse_complement()
#                                 dna_string = str(seqB_vhinit)
#                                 aligned_seq_rev = str(aligned_seq_rev)
#                                 start = re.search(aligned_seq_rev, dna_string).start()
#                                 end = re.search(aligned_seq_rev, dna_string).end()
#                                 quality_vh = quality_vh[start:end]
#                                 quality_score = statistics.mean(quality_vh)
#                                 lowest_quality = min(quality_vh)
#                                 # print(f"{e} | {abx} | {gbid} | {_dfvx_ss.iloc[abx].h3_name} | {gbfp} | {_dfvx_ss.iloc[abx].vh_abi_fp} | {score}| {quality_score} | {lowest_quality}")
#                                 print(f"[+] FILTERED_GOODMATCH: {e}|{abx}|VH|{gbid}|{_dfvx_ss.iloc[abx].h3_name}|{gbfp}|{_dfvx_ss.iloc[abx].vh_abi_fp}|{score}|{quality_score}|{lowest_quality}")if log_msg else None
#                                 filtered_res.append([gbid, _dfvx_ss.iloc[abx].h3_name, gbfp, _dfvx_ss.iloc[abx].vh_abi_fp, score, quality_score, lowest_quality])
#             else:
#                 log.info(f"[+] FILTERED_BADMATCH: {e}|{abx}|VH|{gbid}|{_dfvx_ss.iloc[abx].h3_name}|{gbfp}|{_dfvx_ss.iloc[abx].vh_abi_fp}|{score}|---|---")if log_msg else None
#         except:
#             pass

        # match between gb and vl-abi
        try:          
            for a in align(seqA.seq, seqB_vl, match=par_match, mismatch=par_missmatch, gap_open=par_open, gap_extend=par_extend):  #😄
                score = int(a[2])
                print(f"{e} | {abx} | VL | {_dfvx_ss.iloc[abx].vl_abi_fp} |{score}")
                if score >= int(par_filter_thresh):
                    # getting the aligned seqA and seqB
                    aligned_seq_a = a[0]
                    aligned_seq_a = aligned_seq_a.replace('-', '')
                    aligned_seq_a_start = aligned_seq_a[0:6]
                    aligned_seq_a_length = len(aligned_seq_a) - 6
                    aligned_seq_a_end = aligned_seq_a[aligned_seq_a_length:len(aligned_seq_a)]
                    aligned_seq_b = a[1]
                    
                    # futher filtering of seqB
                    if '-' not in aligned_seq_b:
                        if vl_seq_start_a in aligned_seq_a_start:    # 😄
                            if vl_seq_end_a in aligned_seq_a_end:    # 😄
                                
                                aligned_seq = a[0]
                                aligned_seq = aligned_seq.replace('-', '')
                                aligned_seq = Seq(aligned_seq)
                                aligned_seq_rev = aligned_seq.reverse_complement()
                                dna_string = str(seqB_vlinit)        #😄
                                aligned_seq_rev = str(aligned_seq_rev)
                                start = re.search(aligned_seq_rev, dna_string).start()
                                end = re.search(aligned_seq_rev, dna_string).end()
                                quality_vl = quality_vl[start:end]   #😄
                                quality_score = statistics.mean(quality_vl) #😄
                                lowest_quality = min(quality_vl) #😄
                                # print(f"{e} | {abx} | {gbid} | {_dfvx_ss.iloc[abx].h3_name} | {gbfp} | {_dfvx_ss.iloc[abx].vh_abi_fp} | {score}| {quality_score} | {lowest_quality}")
                                print(f"[+] FILTERED_GOODMATCH: {e}|{abx}|VL|{gbid}|{_dfvx_ss.iloc[abx].h3_name}|{gbfp}|{_dfvx_ss.iloc[abx].vh_abi_fp}|{score}|{quality_score}|{lowest_quality}")if log_msg else None
                                filtered_res.append([gbid, _dfvx_ss.iloc[abx].h3_name, gbfp, _dfvx_ss.iloc[abx].vh_abi_fp, score, quality_score, lowest_quality])
            else:
                log.info(f"[+] FILTERED_BADMATCH: {e}|{abx}|VL|{gbid}|{_dfvx_ss.iloc[abx].h3_name}|{gbfp}|{_dfvx_ss.iloc[abx].vh_abi_fp}|{score}|---|---")if log_msg else None
        except:
            pass
        
        
        
# if not len(filtered_res) == 0:
#     dfres = pd.DataFrame(filtered_res)
#     dfres.columns = ["gbid", "H3_name", "GB_FP", "ABI_FP", "Score", "Quality_score", "Low_quality"]
# else:
#     None

[+] FILTERED_MATCH: outer_index|inner_index|GB_ID_MATCHED|H3_name|GB_FP|ABI_VH_FP|Score|Quality_score|Low_quality
0 | 0 | VL | /Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-A4_A04_VL79.abi |0
1 | 0 | VL | /Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-014-A8_A08_VL79.abi |2
1 | 1 | VL | /Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-014-C4_C04_VL79.abi |4
2 | 0 | VL | /Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-014-A8_A08_VL79.abi |2
2 | 1 | VL | /Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-014-C4_C04_VL79.abi |4
3 | 0 | VL | /Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-F12_VL79.abi |2
3 | 1 | VL | /Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-014-G9_G09_VL79.abi |0
4 | 0 | VL | /Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-F12_VL79.abi |2
4 | 1 | VL | /Users/rp/Desktop/purge/ny-immu

In [69]:
vl_seq_start_a

'CTCCAC'

In [71]:
vl_seq_end_a

'GCTTGG'

In [82]:
def run_alignment_and_filtering(M_gb_abi_vx: pd.DataFrame, 
                                df_vx:pd.DataFrame, 
                                vx_template_gb_dict: dict, 
                                vx_seq_start_a:str, 
                                vx_seq_end_a:str,
                                par_match:int,
                                par_missmatch:int,
                                par_open:int,
                                par_extend:int,
                                par_filter_thresh:int,
                                log :logging, 
                                log_msg: bool=False):
    """
    Returns a dataframe containing gbid,H3_name,GB_FP,ABI_FP,Score,Quality_score,Low_quality
    when for all the datasets where score is above 700
    """
    filtered_res = []
    log.info(f"[+] FILTERED_MATCH: outer_index|inner_index|GB_ID_MATCHED|H3_name|GB_FP|ABI_VH_FP|Score|Quality_score|Low_quality")if log_msg else None
    for e,i in enumerate(range(M_gb_abi_vx.shape[0])[:]):
        gbid = M_gb_abi_vx.iloc[i].gb_id
        gbfp = M_gb_abi_vx.iloc[i].gb_fp
        _dfvx_ss = df_vx[df_vx.h3_name == M_gb_abi_vx.iloc[i].h3_name]   
        for e2, abx in enumerate(range(_dfvx_ss.shape[0])):
            # match between gb and vx-abi
            try:
                seqA = SeqIO.read(vx_template_gb_dict.get(gbid), 'gb')
                seqB = _dfvx_ss.iloc[abx].abi_seq
                seqBinit = _dfvx_ss.iloc[abx].seq_record.seq
                quality = _dfvx_ss.iloc[abx].trimmed_seq_qual
                for a in align(seqA.seq, seqB, match=par_match, mismatch=par_missmatch, gap_open=par_open, gap_extend=par_extend):
                    score = int(a[2])
                    if score >= int(par_filter_thresh):

                        # getting the aligned seqA and seqB
                        aligned_seq_a = a[0]
                        aligned_seq_a = aligned_seq_a.replace('-', '')
                        aligned_seq_a_start = aligned_seq_a[0:6]
                        aligned_seq_a_length = len(aligned_seq_a) - 6
                        aligned_seq_a_end = aligned_seq_a[aligned_seq_a_length:len(aligned_seq_a)]
                        aligned_seq_b = a[1]
                        # futher filtering of seqB
                        if '-' not in aligned_seq_b:
                            if vh_seq_start_a in aligned_seq_a_start:
                                if vh_seq_end_a in aligned_seq_a_end: 
                                    aligned_seq = a[0]
                                    aligned_seq = aligned_seq.replace('-', '')
                                    aligned_seq = Seq(aligned_seq)
                                    aligned_seq_rev = aligned_seq.reverse_complement()
                                    dna_string = str(seqBinit)
                                    aligned_seq_rev = str(aligned_seq_rev)
                                    start = re.search(aligned_seq_rev, dna_string).start()
                                    end = re.search(aligned_seq_rev, dna_string).end()
                                    quality = quality[start:end]
                                    quality_score = statistics.mean(quality)
                                    lowest_quality = min(quality)
                                    # print(f"{e} | {abx} | {gbid} | {_dfvx_ss.iloc[abx].h3_name} | {gbfp} | {_dfvx_ss.iloc[abx].vh_abi_fp} | {score}| {quality_score} | {lowest_quality}")
                                    log.info(f"[+] FILTERED_GOODMATCH: {e}|{abx}|{gbid}|{_dfvx_ss.iloc[abx].h3_name}|{gbfp}|{_dfvx_ss.iloc[abx].vh_abi_fp}|{score}|{quality_score}|{lowest_quality}")if log_msg else None
                                    filtered_res.append([gbid, _dfvx_ss.iloc[abx].h3_name, gbfp, _dfvx_ss.iloc[abx].vh_abi_fp, score, quality_score, lowest_quality])
                else:
                    log.info(f"[+] FILTERED_BADMATCH: {e}|{abx}|{gbid}|{_dfvx_ss.iloc[abx].h3_name}|{gbfp}|{_dfvx_ss.iloc[abx].vh_abi_fp}|{score}|---|---")if log_msg else None
            except:
                pass
    if not len(filtered_res) == 0:
        dfres = pd.DataFrame(filtered_res)
        dfres.columns = ["gbid", "H3_name", "GB_FP", "ABI_FP", "Score", "Quality_score", "Low_quality"]
        return dfres
    else:
        return None

In [64]:
vh_gb_abi_match_filtered = run_alignment_and_filtering(M_gb_abi_vh, df_vh, vh_template_gb_dict, vh_seq_start_a, vh_seq_end_a, par_match, par_missmatch, par_open, par_extend, par_filter_thresh, log, log_msg=False)

In [61]:
vh_gb_abi_match_filtered

,gbid,H3_name,GB_FP,ABI_FP,Score,Quality_score,Low_quality
0,VH-TMH577-hF-004-G09,TMH577-hF-004-G09,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,782,60.792839,43
1,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,842,61.114014,43
2,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,842,61.114014,43
3,VH-TMH577-hF-012-B03,TMH577-hF-012-B03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,812,60.709360,37
4,VH-TMH577-hF-012-B03,TMH577-hF-012-B03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,812,60.709360,37
...,...,...,...,...,...,...,...
298,VH-TMH577-hF-017-G10,TMH577-hF-017-G10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,770,60.821530,32
299,VH-TMH577-hF-017-G10,TMH577-hF-017-G10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,770,60.821530,32
300,VH-TMH577-hF-017-H03,TMH577-hF-017-H03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,770,60.742857,33
301,VH-TMH577-hF-017-H03,TMH577-hF-017-H03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,770,60.742857,33


In [72]:
vl_gb_abi_match_filtered = run_alignment_and_filtering(M_gb_abi_vl, df_vl, vl_template_gb_dict, vl_seq_start_a, vl_seq_end_a, par_match, par_missmatch, par_open, par_extend, par_filter_thresh, log, log_msg=True)

In [78]:
isinstance(vl_gb_abi_match_filtered, type(None))

True

In [83]:
vh_gb_abi_match_filtered.loc[5].at['ABI_FP']

'/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-014-E1_E01_GATC-VH60-2617917.abi'

In [46]:
# save dataframe to an excel sheet

In [47]:
# with pd.ExcelWriter('final_results.xlsx', engine='xlsxwriter') as writer:
    # df_vh.to_excel(writer, sheet_name='H3_probe_matched_VH_and_VL')
    # res_df_copy.to_excel(writer, sheet_name='VH and VL Copied Files')
    # M_gb_abi_vh.to_excel(writer, sheet_name='ID matched gb and vh abi')
    # M_gb_abi_vh.to_excel(writer, sheet_name='ID matched gb and vh abi')